In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
api_key=os.getenv('api_key')

Part 1 : Reading the Data from Docx or PDF

In [3]:
from docx import Document
def docx_read(file_path):
    text1=[]
    doc=Document(file_path)
    for para in doc.paragraphs:
        text1.append(para.text)
    return "\n".join(text1)

In [4]:
from PyPDF2 import PdfReader
def pdf_read(file_path):
    text2=""
    pdf=PdfReader(file_path)
    for page in pdf.pages:
        text2 += page.extract_text()
    return text2

In [5]:
def resume_parse(file_path):
    if file_path.endswith(".docx"):
        return docx_read(file_path)
    elif file_path.endswith(".pdf"):
        return pdf_read(file_path)
    else:
        return ValueError("Unsupported file format. Upload either a PDF or DOCX file.")

Part 2: Uploading the file


In [6]:
import streamlit as st
uploaded_file = st.file_uploader("Upload a PDF or DOC file", type=["pdf", "docx"])


2024-12-10 21:12:55.251 
  command:

    streamlit run C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [7]:
resume_text=resume_parse(uploaded_file)

AttributeError: 'NoneType' object has no attribute 'endswith'

In [7]:
resume_chunks=[]
for chunk in resume_text.split("\n"):
    if chunk.strip():
        resume_chunks.append(chunk)

Part 3: Embeddings and Storage

In [9]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings=GoogleGenerativeAIEmbeddings(google_api_key=api_key,model="models/embedding-001")
vector=embeddings.embed_documents(resume_chunks)

In [ ]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

index = faiss.IndexFlatL2(len(vector[0]))

vector_store = FAISS(
    embedding_function=embeddings,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
    index=index
)

vector_store.add_texts(resume_chunks)

Part 4: Retrieval and Query Input

In [11]:
from langchain.prompts import PromptTemplate
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough

llm = GoogleGenerativeAI(google_api_key=api_key,model="gemini-1.5-flash")


prompt_template = PromptTemplate(
    template="You are an ATS system. Based on the context: {context}, answer the question: {question}",
    input_variables=["context", "question"]
    
)

retriever = vector_store.as_retriever()

qa_chain = (
    {"context": retriever, "question": RunnablePassthrough()} #allows user's query
    | prompt_template
    | llm
)


Part 5: Taking Input and Generating Response

In [13]:
query=input("Enter Query: ")

response=qa_chain.invoke(query)
print(response)

Based solely on the provided snippets, I would give a moderate ATS score.  While the snippets demonstrate quantitative achievements (CGPA, chatbot accuracy, RAG system accuracy), they lack crucial context.  The ATS would likely struggle to fully assess the resume because:

* **Missing Job Titles/Roles:**  There's no indication of what roles these accomplishments relate to.  An ATS needs to match skills and experiences to specific job requirements, and without job titles, this matching is severely hampered.
* **Lack of Keywords:**  While numbers are present, relevant keywords related to specific job functions (e.g., "Python," "Natural Language Processing," "Machine Learning") are absent.  This limits the ATS's ability to identify relevant skills.
* **Incomplete Picture:** Four snippets are insufficient to provide a comprehensive view of the candidate's skills and experience.  A full resume is needed for accurate scoring.


Therefore, a preliminary ATS score would be somewhere in the **5